<a href="https://colab.research.google.com/github/aryanc55/LTFSDSFinHack3/blob/main/Copy_of_FeatureEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret
!pip install category_encoders
!pip install imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from functools import reduce
from datetime import datetime, timedelta
from pycaret.classification import *
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g5JAkhgA8KxO5HrvTbhCWhkwkBAIqVgLr3czQYpXOvCwxhnp6GcIsk
Mounted at /content/drive


In [ ]:
train_b = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Train/train_bureau.xlsx')

In [ ]:
train_d = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Train/train_Data.xlsx')

In [ ]:
train_b.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaN,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaN,2019-12-19,NaN,"44,000","1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaN,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN
3,1,True,PRIMARY,Auto Loan (Personal),NBF,2017-09-30,Individual,Closed,2013-09-27,2017-09-21 00:00:00,NaT,NaN,"3,00,000",NaN,0,NaN,0,0.0,NaN,"20170930,20170801,20170731,20170630,20170531,2...",000DDD0270260270260270240270270000320000000000...,"0,,15925,23754,31494,39147,46713,54194,61590,6...","0,,1014,1014,1014,1014,1014,1014,1014,983,0,92...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,1,True,PRIMARY,Tractor Loan,NBF,2016-02-29,Individual,Closed,2012-02-10,2016-02-01 00:00:00,NaT,NaN,"2,75,000",NaN,0,NaN,0,0.0,NaN,"20160229,20160131,20151231,20151130,20151031,2...",0000000000000000000000000000000000000000000000...,"0,0,23658,23321,22989,46321,45662,45012,68030,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN


In [ ]:
test_d = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Test/test_Data.xlsx')

In [ ]:
test_b = pd.read_excel('/content/drive/MyDrive/LTFSDSFinHack3/Test/test_bureau.xlsx')

In [ ]:
test_b.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,NaN,589,NaN,0,0.0,NaN,"20191117,",000,"589,","0,","0,",NaN
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,"25,00,000","30,797/Monthly","21,07,227",F03,0,0.0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120.0
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,"99,000","1,223/Monthly","83,458",F03,0,0.0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120.0
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,"3,65,000",NaN,0,NaN,0,0.0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",NaN
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,"25,801",NaN,0,NaN,0,0.0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",NaN


In [146]:
train_bb = train_b.copy()

In [147]:
test_bb = test_b.copy()

In [148]:
# Data Treatment:
#     -categorical vars

In [149]:
# id -dont do it
# self indiactor -nochange
# match type- not req
# acct type -required 3 columns chosen for remaning crosstab then combine cols
    
 #contributor type ARC is not ptesent in test so crosstab and then delete arc   
# ownership indicator no rpoblem
# account status not present in test Sold/Purchased ,Cancelled cross tab then delete                            
# credit limit CREDIT-LIMIT/SANC AMT  DISBURSED-AMT/HIGH CREDIT kbindiscretization
# installment not present in test data F04 F07  F08 crosstab then delete it
# asset class 2 not ptresent in test crosstab and remove it

In [150]:
train_b.ID

0              1
1              1
2              1
3              1
4              1
           ...  
560839    143393
560840    143393
560841    143393
560842    143394
560843    143395
Name: ID, Length: 560844, dtype: int64

In [151]:
train_SELF_INDICATOR=pd.crosstab(train_b.ID,train_b['SELF-INDICATOR'],dropna= True).reset_index()
test_SELF_INDICATOR=pd.crosstab(test_b.ID,test_b['SELF-INDICATOR'],dropna= True).reset_index()

In [152]:
train_MATCH_TYPE=pd.crosstab(train_b.ID,train_b['MATCH-TYPE'],dropna= True).reset_index()
test_MATCH_TYPE=pd.crosstab(test_b.ID,test_b['MATCH-TYPE'],dropna= True).reset_index()

In [153]:
x = set(train_bb['ACCT-TYPE'].unique())  
y = set(test_bb['ACCT-TYPE'].unique()) 
common = x.intersection(y)
print(list(common)) #now decide cols to keep 

['Fleet Card', 'Commercial Equipment Loan', 'Business Non-Funded Credit Facility-Priority Sector- Small Business', 'Auto Loan (Personal)', 'Construction Equipment Loan', 'Pradhan Mantri Awas Yojana - CLSS', 'Microfinance Others', 'Business Loan General', 'Loan Against Bank Deposits', 'Personal Loan', 'Secured Credit Card', 'Non-Funded Credit Facility', 'Business Loan Priority Sector  Others', 'Corporate Credit Card', 'Loan to Professional', 'Microfinance Housing Loan', 'Mudra Loans   Shishu / Kishor / Tarun', 'JLG Individual', 'Property Loan', 'Kisan Credit Card', 'Commercial Vehicle Loan', 'Business Loan Priority Sector  Agriculture', 'Business Loan - Secured', 'Business Non-Funded Credit Facility General', 'Prime Minister Jaan Dhan Yojana - Overdraft', 'Loan Against Shares / Securities', 'Business Non-Funded Credit Facility-Priority Sector-Agriculture', 'Microfinance Business Loan', 'Microfinance Personal Loan', 'Overdraft', 'Two-Wheeler Loan', 'Other', 'Business Loan Against Bank De

In [154]:
train_ACCT_TYPE=pd.crosstab(train_b.ID,train_b['ACCT-TYPE'],dropna= True).reset_index()
test_ACCT_TYPE=pd.crosstab(test_b.ID,test_b['ACCT-TYPE'],dropna= True).reset_index()

cols_to_keep =['Tractor Loan','Gold Loan','Business Loan Priority Sector  Agriculture']

train_ACCT_TYPE["Others"] = pd.DataFrame(train_ACCT_TYPE.drop(columns=cols_to_keep).sum(axis=1))
test_ACCT_TYPE["Others"] = pd.DataFrame(test_ACCT_TYPE.drop(columns=cols_to_keep).sum(axis=1))
filter_cols =cols_to_keep + ['Others','ID'] 
train_ACCT_TYPE = train_ACCT_TYPE.filter(filter_cols)
test_ACCT_TYPE = test_ACCT_TYPE.filter(filter_cols)

In [155]:
train_CONTRIBUTOR_TYPE=pd.crosstab(train_b.ID,train_b['CONTRIBUTOR-TYPE'],dropna= True).drop(columns = ['ARC']).reset_index()
test_CONTRIBUTOR_TYPE=pd.crosstab(test_b.ID,test_b['CONTRIBUTOR-TYPE'],dropna= True).reset_index()


In [156]:
train_OWNERSHIP_IND=pd.crosstab(train_b.ID,train_b['OWNERSHIP-IND'],dropna= True).reset_index()
test_OWNERSHIP_IND=pd.crosstab(test_b.ID,test_b['OWNERSHIP-IND'],dropna= True).reset_index()


In [157]:
train_ACCOUNT_STATUS=pd.crosstab(train_b.ID,train_b['ACCOUNT-STATUS'],dropna= True).drop(columns=['Sold/Purchased','Cancelled']).reset_index()
test_ACCOUNT_STATUS=pd.crosstab(test_b.ID,test_b['ACCOUNT-STATUS'],dropna= True).reset_index()


In [158]:
train_INSTALLMENT_FREQUENCY=pd.crosstab(train_b.ID,train_b['INSTALLMENT-FREQUENCY'],dropna= True).drop(columns=['F04','F07','F08']).reset_index()
test_INSTALLMENT_FREQUENCY=pd.crosstab(test_b.ID,test_b['INSTALLMENT-FREQUENCY'],dropna= True).reset_index()

In [159]:
train_ASSET_CLASS=pd.crosstab(train_b.ID,train_b['ASSET_CLASS'],dropna= True).drop(columns=['2']).reset_index()
test_ASSET_CLASS=pd.crosstab(test_b.ID,test_b['ASSET_CLASS'],dropna= True).reset_index()

In [160]:
def onehotencoder(train,test,cols=[]):
    
    from sklearn.preprocessing import OneHotEncoder
    #object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    object_cols=cols
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    

    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

def targetencoding(train,test,y_train,drop=False,cols=[]):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = cols
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))

    if drop :
        traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
        testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc


def ordinalencoding(train,test,y_train,mapping,drop=False,cols=[]):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = cols
    print(f'ordinal encoding for features {cat_features}')
    target_enc = ce.OrdinalEncoder(cols=cat_features,mapping = mapping)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_ordinal'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_ordinal'))

    if drop :
        traintrgtenc = traintrgtenc.drop(cat_features, axis =1)
        testtrgtenc = testtrgtenc.drop(cat_features, axis =1)

    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc

In [161]:
train_bb['DISBURSED-AMT/HIGH CREDIT'] = train_b['DISBURSED-AMT/HIGH CREDIT'].str.replace(',', '').astype('float')
test_bb['DISBURSED-AMT/HIGH CREDIT'] = test_b['DISBURSED-AMT/HIGH CREDIT'].str.replace(',', '').astype('float')

In [162]:
train_bb['CURRENT-BAL'] = train_b['CURRENT-BAL'].fillna(0)
test_bb['CURRENT-BAL'] = test_b['CURRENT-BAL'].fillna(0)
train_bb['CURRENT-BAL'] = train_b['CURRENT-BAL'].str.replace(',', '').astype('float')
test_bb['CURRENT-BAL'] = test_b['CURRENT-BAL'].str.replace(',', '').astype('float')

In [163]:

train_CURRENT = train_bb[['CURRENT-BAL','ID']].dropna()
test_CURRENT = test_bb[['CURRENT-BAL','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['CURRENT-BAL'].dropna().values.reshape(-1,1))

train_CURRENT['CURRENT-BAL'] = enc.transform(train_CURRENT['CURRENT-BAL'].values.reshape(-1,1))
test_CURRENT['CURRENT-BAL']= enc.transform(test_CURRENT['CURRENT-BAL'].values.reshape(-1,1))

In [164]:
train_CURRENT=pd.crosstab(train_CURRENT.ID,train_CURRENT['CURRENT-BAL'],dropna= True).reset_index()
test_CURRENT=pd.crosstab(test_CURRENT.ID,test_CURRENT['CURRENT-BAL'],dropna= True).reset_index()
test_CURRENT['3.0']=0

In [165]:

train_DISBURSED = train_bb[['DISBURSED-AMT/HIGH CREDIT','ID']].dropna()
test_DISBURSED = test_bb[['DISBURSED-AMT/HIGH CREDIT','ID']].dropna()

enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
enc.fit(train_bb['DISBURSED-AMT/HIGH CREDIT'].dropna().values.reshape(-1,1))

train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'] = enc.transform(train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'].values.reshape(-1,1))
test_DISBURSED['DISBURSED-AMT/HIGH CREDIT']= enc.transform(test_DISBURSED['DISBURSED-AMT/HIGH CREDIT'].values.reshape(-1,1))

In [166]:
train_DISBURSED=pd.crosstab(train_DISBURSED.ID,train_DISBURSED['DISBURSED-AMT/HIGH CREDIT'],dropna= True).reset_index()
test_DISBURSED=pd.crosstab(test_DISBURSED.ID,test_DISBURSED['DISBURSED-AMT/HIGH CREDIT'],dropna= True).reset_index()
test_DISBURSED['2.0']=0

In [167]:
# #discretisation baes on clusters

# train_tenure = train_bb[['TENURE','ID']].dropna()
# test_tenure = test_bb[['TENURE','ID']].dropna()

# enc = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='kmeans')
# enc.fit(train_bb['TENURE'].dropna().values.reshape(-1,1))

# train_tenure['TENURE'] = enc.transform(train_tenure['TENURE'].values.reshape(-1,1))
# test_tenure['TENURE']= enc.transform(test_tenure['TENURE'].values.reshape(-1,1))

In [168]:
# train_tenure=pd.crosstab(train_tenure.ID,train_tenure['TENURE'],dropna= True).reset_index()
# test_tenure=pd.crosstab(test_tenure.ID,test_tenure['TENURE'],dropna= True).reset_index()


In [169]:
# train_SELF_INDICATOR
# train_INSTALLMENT_FREQUENCY
# train_ACCOUNT_STATUS
# train_OWNERSHIP_IND
# train_CONTRIBUTOR_TYPE
# train_ACCT_TYPE

In [170]:
dataframes = [train_b.ID,train_SELF_INDICATOR,train_INSTALLMENT_FREQUENCY,train_ACCOUNT_STATUS,train_OWNERSHIP_IND,
              train_CONTRIBUTOR_TYPE,train_ACCT_TYPE,train_MATCH_TYPE,train_ASSET_CLASS,train_CURRENT,train_DISBURSED]
final_df_train_beauro = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

final_df_train_beauro = final_df_train_beauro.fillna(0)

In [171]:
dataframes =  [train_b.ID,train_SELF_INDICATOR,train_INSTALLMENT_FREQUENCY,train_ACCOUNT_STATUS,train_OWNERSHIP_IND,
              train_CONTRIBUTOR_TYPE,train_ACCT_TYPE,train_MATCH_TYPE,train_ASSET_CLASS,train_CURRENT,train_DISBURSED]
final_df_test_beauro = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)
final_df_test_beauro = final_df_test_beauro.fillna(0)

In [106]:
###### now train data 

In [172]:
train_dd = train_d.copy()
test_dd = test_d.copy()

In [173]:
train_dd.head()

,ID,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Area,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,DisbursalDate,MaturityDAte,AuthDate,AssetID,ManufacturerID,SupplierID,LTV,SEX,AGE,MonthlyIncome,City,State,ZiPCODE,Top-up Month
0,1,Monthly,Arrear,Closed,PDC_E,1,NaN,48,450000,275000.0,275000.0,24000.0,2012-02-10,2016-01-15,2012-02-10,4022465,1568,21946,61.11,M,49.0,35833.33,RAISEN,MADHYA PRADESH,464993.0,> 48 Months
1,2,Monthly,Advance,Closed,PDC,333,BHOPAL,47,485000,350000.0,350000.0,10500.0,2012-03-31,2016-02-15,2012-03-31,4681175,1062,34802,70.00,M,23.0,666.67,SEHORE,MADHYA PRADESH,466001.0,No Top-up Service
2,3,Quatrly,Arrear,Active,Direct Debit,1,NaN,68,690000,519728.0,519728.0,38300.0,2017-06-17,2023-02-10,2017-06-17,25328146,1060,127335,69.77,M,39.0,45257.00,BHOPAL,MADHYA PRADESH,462030.0,12-18 Months
3,7,Monthly,Advance,Closed,Billed,125,GUNA,48,480000,400000.0,400000.0,11600.0,2013-11-29,2017-11-10,2013-11-29,13021591,1060,25094,80.92,M,24.0,20833.33,ASHOK NAGAR,MADHYA PRADESH,473335.0,> 48 Months
4,8,Monthly,Arrear,Closed,Billed,152,BILASPUR,44,619265,440000.0,440000.0,15000.0,2011-12-08,2015-07-05,2011-12-08,3291320,1046,21853,71.05,M,56.0,27313.67,BILASPUR,CHATTISGARH,495442.0,36-48 Months


In [174]:
train_dd['SEX'] = train_dd['SEX'].fillna(train_dd['SEX'].mode()[0])
test_dd['SEX'] = test_dd['SEX'].fillna(test_dd['SEX'].mode()[0])

train_dd['AGE'] = train_dd['AGE'].fillna(train_dd['AGE'].mode()[0])
test_dd['AGE'] = test_dd['AGE'].fillna(test_dd['AGE'].mode()[0])

train_dd['MonthlyIncome'] = train_dd['MonthlyIncome'].fillna(train_dd['MonthlyIncome'].mode()[0])
test_dd['MonthlyIncome'] = test_dd['MonthlyIncome'].fillna(test_dd['MonthlyIncome'].mode()[0])

train_dd['ZiPCODE']  =train_dd['ZiPCODE'].fillna(method = 'ffill').astype('int') 
test_dd['ZiPCODE']  =test_dd['ZiPCODE'].fillna(method = 'ffill').astype('int') 


train_dd =train_dd.fillna(method ='ffill') 
test_dd =test_dd.fillna(method = 'ffill') 

In [175]:
train_dd['AuthDate'] = pd.to_datetime(train_d['AuthDate'] ,errors='coerce')
test_dd['AuthDate'] = pd.to_datetime(test_d['AuthDate'] ,errors='coerce')

train_dd['DisbursalDate'] = pd.to_datetime(train_d['DisbursalDate'] ,errors='coerce')
test_dd['DisbursalDate'] = pd.to_datetime(test_d['DisbursalDate'] ,errors='coerce')

train_dd['MaturityDAte'] = pd.to_datetime(train_d['MaturityDAte'] ,errors='coerce')
test_dd['MaturityDAte'] = pd.to_datetime(test_d['MaturityDAte'] ,errors='coerce')

train_dd = train_d.dropna()
#test_dd['MaturityDAte'] = test_d['MaturityDAte'].fillna(method='ffill')

In [176]:
train_dd['AuthtoMatureDays'] = (train_dd['AuthDate'] - train_dd['MaturityDAte']).astype('int')
test_dd['AuthtoMatureDays'] = (test_dd['AuthDate'] - test_dd['MaturityDAte']).astype('int')

train_dd['AuthtoDisDays'] = (train_dd['AuthDate'] - train_dd['DisbursalDate']).astype('int')
test_dd['AuthtoDisDays'] = (test_dd['AuthDate'] - test_dd['DisbursalDate']).astype('int')

In [177]:
train_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107126 entries, 1 to 128654
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ID                107126 non-null  int64         
 1   Frequency         107126 non-null  object        
 2   InstlmentMode     107126 non-null  object        
 3   LoanStatus        107126 non-null  object        
 4   PaymentMode       107126 non-null  object        
 5   BranchID          107126 non-null  int64         
 6   Area              107126 non-null  object        
 7   Tenure            107126 non-null  int64         
 8   AssetCost         107126 non-null  int64         
 9   AmountFinance     107126 non-null  float64       
 10  DisbursalAmount   107126 non-null  float64       
 11  EMI               107126 non-null  float64       
 12  DisbursalDate     107126 non-null  datetime64[ns]
 13  MaturityDAte      107126 non-null  datetime64[ns]
 14  Auth

In [178]:
train_dd['AuthYear'] = train_dd['AuthDate'].dt.year
test_dd['AuthYear'] = test_dd['AuthDate'].dt.year 
train_dd['AuthMonth'] = train_dd['AuthDate'].dt.month 
test_dd['AuthMonth'] = test_dd['AuthDate'].dt.month  


train_dd['MaturityYear'] = train_d['MaturityDAte'].dt.year  
test_dd['MaturityYear'] = test_d['MaturityDAte'].dt.year  

train_dd['MaturityMonth'] = train_d['MaturityDAte'].dt.month  
test_dd['MaturityMonth'] = test_d['MaturityDAte'].dt.month  

In [179]:
train_dd.isna().sum()
#test_dd.isna().sum()

ID                  0
Frequency           0
InstlmentMode       0
LoanStatus          0
PaymentMode         0
BranchID            0
Area                0
Tenure              0
AssetCost           0
AmountFinance       0
DisbursalAmount     0
EMI                 0
DisbursalDate       0
MaturityDAte        0
AuthDate            0
AssetID             0
ManufacturerID      0
SupplierID          0
LTV                 0
SEX                 0
AGE                 0
MonthlyIncome       0
City                0
State               0
ZiPCODE             0
Top-up Month        0
AuthtoMatureDays    0
AuthtoDisDays       0
AuthYear            0
AuthMonth           0
MaturityYear        0
MaturityMonth       0
dtype: int64

In [180]:
train_dd = train_dd.drop(columns=['City','Area','AuthDate','DisbursalDate','MaturityDAte'])
test_dd = test_dd.drop(columns=['City','Area','AuthDate','DisbursalDate','MaturityDAte'])

In [181]:
mapping_dict = {'No Top-up Service':0, '12-18 Months':1,'18-24 Months':2, '24-30 Months':3, 
                '30-36 Months':4,'36-48 Months':5,' > 48 Months':6}

In [182]:
train_dd['Top-up Month'] = train_dd['Top-up Month'].replace(mapping_dict) 

In [183]:
mapping_dict_f= [
{'col': 'Frequency', 'mapping': {'BI-Monthly':0, 'Monthly': 1, 'Quatrly': 2,  'Half Yearly': 3}}
]

In [184]:
f1,f2 = ordinalencoding(train_dd,test_dd,train_dd['Top-up Month'],drop =True,cols=['Frequency'],mapping= mapping_dict_f)

ordinal encoding for features ['Frequency']
(107126, 27) (14745, 26)


In [185]:
f11,f22 = onehotencoder(f1,f2,cols=['InstlmentMode','LoanStatus','SEX'])

['InstlmentMode', 'LoanStatus', 'SEX']
(107126, 30) (14745, 29)


In [186]:
f111,f222 = targetencoding(f11,f22,f11['Top-up Month'],drop=True,cols=['PaymentMode','State','BranchID','ZiPCODE','ManufacturerID']) #because branch id and area same

targest emcoding for features ['PaymentMode', 'State', 'BranchID', 'ZiPCODE', 'ManufacturerID']
(107126, 30) (14745, 29)


In [187]:

# final_df_train_beauro.rename(columns={2.0: '2.0', 3.0: '3.0'}, inplace=True)

# final_df_test_beauro.rename(columns={2.0: '2.0', 3.0: '3.0'}, inplace=True)

In [188]:
dataframes = [f111,final_df_train_beauro]
final_df_train = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [189]:
dataframes = [f222,final_df_test_beauro]
final_df_test = reduce(lambda left, right: pd.merge(left, right, on='ID', how='left'), dataframes)

In [190]:
final_df_test.isna().sum()

ID                     0
Tenure                 0
AssetCost              0
AmountFinance          0
DisbursalAmount        0
                   ...  
3.0_x              14745
0.0                14745
1.0                14745
2.0_y              14745
3.0_y              14745
Length: 83, dtype: int64

In [191]:
final_df_train = final_df_train.fillna(0)
final_df_test = final_df_test.fillna(0)

In [192]:
final_df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471743 entries, 0 to 471742
Data columns (total 84 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   ID                                          471743 non-null  int64  
 1   Tenure                                      471743 non-null  int64  
 2   AssetCost                                   471743 non-null  int64  
 3   AmountFinance                               471743 non-null  float64
 4   DisbursalAmount                             471743 non-null  float64
 5   EMI                                         471743 non-null  float64
 6   AssetID                                     471743 non-null  int64  
 7   SupplierID                                  471743 non-null  int64  
 8   LTV                                         471743 non-null  float64
 9   AGE                                         471743 non-null  float64
 

In [193]:
final_df_train.columns

Index([                                        'ID',
                                           'Tenure',
                                        'AssetCost',
                                    'AmountFinance',
                                  'DisbursalAmount',
                                              'EMI',
                                          'AssetID',
                                       'SupplierID',
                                              'LTV',
                                              'AGE',
                                    'MonthlyIncome',
                                     'Top-up Month',
                                 'AuthtoMatureDays',
                                    'AuthtoDisDays',
                                         'AuthYear',
                                        'AuthMonth',
                                     'MaturityYear',
                                    'MaturityMonth',
                                'Frequency_ord

In [194]:
set(final_df_train.columns) - set(final_df_test.columns)

{'Top-up Month'}

In [ ]:
cols = ['Tenure', 'AssetID','SupplierID', 'AGE',  'AuthYear', 'AuthMonth','MaturityYear','MaturityMonth', 'Frequency_ordinal','InstlmentMode_Advance','InstlmentMode_Arrear',
                                'LoanStatus_Active',
                                'LoanStatus_Closed', 'SEX_F','SEX_M','PaymentMode_target','State_target','BranchID_target','ZiPCODE_target','ManufacturerID_target',
                                          'False_x', 'True_x',
                                              'F01', 'F02','F03','F05','F06', 'F10',
                                           'Active', 'Closed','Delinquent', 'Restructured', 'SUIT FILED (WILFUL DEFAULT)','Settled', 'Suit Filed','WILFUL DEFAULT',
                                      'Written Off', 'Guarantor', 'Individual','Joint','Primary', 'Supl Card Holder',
                                              'CCC', 'COP', 'FRB', 'HFC', 'MFI', 'NAB',  'NBF', 'OFI', 'PRB', 'RRB', 'SFB',
                                     'Tractor Loan',
                                        'Gold Loan',
       'Business Loan Priority Sector  Agriculture',
                                           'Others',
                                          'PRIMARY',
                                        'SECONDARY',
                                               '01',
                                                '1',
                                         'Doubtful',
                                             'Loss',
                          'Special Mention Account',
                                         'Standard',
                                      'SubStandard',
                                            '0.0_y',
                                            '1.0_y',
                                            '2.0_x',
                                            '3.0_x',
                                            '0.0_x',
                                            '1.0_x',
                                            '2.0_y',
                                            '3.0_y'  ]

In [195]:
final_df_train.to_csv('/content/drive/MyDrive/LTFSDSFinHack3/train.csv',index =False)
final_df_test.to_csv('/content/drive/MyDrive/LTFSDSFinHack3/test.csv',index =False)

In [ ]:
# data, target, train_size = 0.7, sampling = True, sample_estimator = None, categorical_features = None, 
# categorical_imputation = ‘constant’, ordinal_features = None, high_cardinality_features = None, high_cardinality_method = ‘frequency’,
#  numeric_features = None, numeric_imputation = ‘mean’, date_features = None, ignore_features = None, normalize = False, normalize_method = ‘zscore’, 
#  transformation = False, transformation_method = ‘yeo-johnson’, handle_unknown_categorical = True, unknown_categorical_method = ‘least_frequent’, 
#  pca = False, pca_method = ‘linear’, pca_components = None, ignore_low_variance = False, combine_rare_levels = False, rare_level_threshold = 0.10, 
#  bin_numeric_features = None, remove_outliers = False, outliers_threshold = 0.05, remove_multicollinearity = False, multicollinearity_threshold = 0.9,
#   remove_perfect_collinearity = False, create_clusters = False, cluster_iter = 20, polynomial_features = False, polynomial_degree = 2, 
#   trigonometry_features = False, polynomial_threshold = 0.1, group_features = None, group_names = None, feature_selection = False, 
#   feature_selection_threshold = 0.8, feature_interaction = False, feature_ratio = False, interaction_threshold = 0.01, fix_imbalance = False, 
#   fix_imbalance_method = None, data_split_shuffle = True, folds_shuffle = False, n_jobs = -1, html = True, session_id = None, log_experiment = False, 
#   experiment_name = None, 
# log_plots = False, log_profile = False, log_data = False, silent=False, verbose=True, profile = False

In [ ]:
pycaret = setup(data =final_df_train , target ='Top-up Month',remove_perfect_collinearity = False,fix_imbalance = True,fix_imbalance_method=RandomUnderSampler(random_state=0),session_id=123) #feature_selection,remove_perfect_collinearity

,Description,Value
0,session_id,123
1,Target,Top-up Month
2,Target Type,Multiclass
3,Label Encoded,"0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6"
4,Original Data,"(471743, 84)"
5,Missing Values,False
6,Numeric Features,52
7,Categorical Features,31
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
xgboost = create_model('xgboost', tree_method = 'gpu_hist', gpu_id = 0)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7100,0.9325,0.8478,0.8173,0.7252,0.5558,0.5940
1,0.7078,0.9298,0.8373,0.8141,0.7237,0.5514,0.5884
2,0.7107,0.9307,0.8472,0.8172,0.7262,0.5562,0.5938
3,0.7075,0.9320,0.8450,0.8166,0.7232,0.5530,0.5914
4,0.7043,0.9330,0.8421,0.8155,0.7201,0.5492,0.5882
5,0.7067,0.9297,0.8374,0.8138,0.7226,0.5503,0.5876
6,0.7085,0.9313,0.8425,0.8166,0.7239,0.5540,0.5923
7,0.7111,0.9317,0.8453,0.8174,0.7267,0.5568,0.5941
8,0.7094,0.9338,0.8450,0.8169,0.7245,0.5553,0.5936
9,0.7126,0.9305,0.8441,0.8157,0.7279,0.5572,0.5935


In [ ]:
rf = create_model(estimator = 'rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7471,0.9541,0.8969,0.8325,0.7576,0.6036,0.6379
1,0.7428,0.9521,0.8930,0.8306,0.7538,0.5981,0.6330
2,0.7436,0.9513,0.8891,0.8285,0.7546,0.5976,0.6311
3,0.7427,0.9532,0.8889,0.8296,0.7537,0.5973,0.6318
4,0.7403,0.9530,0.8877,0.8288,0.7515,0.5943,0.6291
5,0.7363,0.9490,0.8829,0.8238,0.7475,0.5878,0.6220
6,0.7394,0.9514,0.8877,0.8283,0.7505,0.5934,0.6284
7,0.7399,0.9511,0.8892,0.8278,0.7511,0.5938,0.6281
8,0.7456,0.9535,0.8940,0.8311,0.7564,0.6013,0.6351
9,0.7440,0.9494,0.8912,0.8292,0.7550,0.5986,0.6321


In [ ]:
# blend_specific_hard = blend_models(estimator_list = [tune_rf,tune_ridge,tune_lgb], method = 'hard') 

In [ ]:
# blend_specific_soft = blend_models(estimator_list = [tune_rf,tune_et,tune_lgb], method = 'soft') #those who support predict prob 

In [ ]:
# plot_model(estimator=tune_et, plot='feature')

In [ ]:
# plot_model(tune_et, plot = 'confusion_matrix')

In [76]:
final_rf = finalize_model(rf)

In [196]:
prediction_rf = predict_model(estimator= final_rf, data=final_df_test,verbose=True)

In [197]:
prediction_rf

,ID,Tenure,AssetCost,AmountFinance,DisbursalAmount,EMI,AssetID,SupplierID,LTV,AGE,MonthlyIncome,AuthtoMatureDays,AuthtoDisDays,AuthYear,AuthMonth,MaturityYear,MaturityMonth,Frequency_ordinal,InstlmentMode_Advance,InstlmentMode_Arrear,LoanStatus_Active,LoanStatus_Closed,SEX_F,SEX_M,PaymentMode_target,State_target,BranchID_target,ZiPCODE_target,ManufacturerID_target,False_x,True_x,F01,F02,F03,F05,F06,F10,Active,Closed,Delinquent,...,Written Off,Guarantor,Individual,Joint,Primary,Supl Card Holder,CCC,COP,FRB,HFC,MFI,NAB,NBF,OFI,PRB,RRB,SFB,Tractor Loan,Gold Loan,Business Loan Priority Sector Agriculture,Others,PRIMARY,SECONDARY,01,1,Doubtful,Loss,Special Mention Account,Standard,SubStandard,0.0_y,1.0_y,2.0_x,3.0_x,0.0,1.0,2.0_y,3.0_y,Label,Score
0,4,46,480000,365000.0,365000.0,1000.0,3524747,22354,75.83,50.0,32069.00,-118886400000000000,0,2011,12,2015,10,1,1.0,0.0,0.0,1.0,0.0,1.0,0.682510,0.923196,0.783526,0.783526,0.733988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.69
1,5,45,480000,285000.0,285000.0,9300.0,4985862,22354,57.44,35.0,25000.00,-116035200000000000,0,2012,4,2016,1,1,1.0,0.0,0.0,1.0,0.0,1.0,0.816968,0.923196,0.783526,0.783526,0.733988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.76
2,6,48,580000,400000.0,400000.0,35800.0,12881783,65929,68.97,37.0,23333.33,-122601600000000000,0,2013,10,2017,9,2,0.0,1.0,0.0,1.0,0.0,1.0,0.816968,0.923196,0.783526,0.783526,0.732945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.83
3,25,36,725000,500000.0,500000.0,52000.0,10613776,68597,61.79,55.0,91666.67,-85968000000000000,0,2013,5,2016,2,3,0.0,1.0,0.0,1.0,0.0,1.0,0.812822,0.513832,0.783526,0.941176,0.783526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.72
4,119,48,617000,400000.0,400000.0,35400.0,7911110,38493,59.09,48.0,12500.00,-126662400000000000,0,2012,11,2016,12,2,0.0,1.0,0.0,1.0,0.0,1.0,0.816968,0.913290,0.536232,0.037159,0.783526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14740,143396,35,530016,419616.0,419616.0,14723.0,4276747,22094,79.17,37.0,33333.33,-93052800000000000,0,2012,2,2015,2,1,0.0,1.0,0.0,1.0,0.0,1.0,0.816968,0.731367,0.686022,0.862500,0.733988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.82
14741,143397,36,595000,446500.0,446500.0,15491.0,4746586,61196,75.04,37.0,33333.33,-94176000000000000,0,2012,4,2015,4,1,0.0,1.0,0.0,1.0,0.0,1.0,0.816968,0.799343,0.479853,0.783526,0.783526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.77
14742,143398,36,595000,446500.0,446500.0,15491.0,4746588,61196,75.04,37.0,33333.33,-94176000000000000,0,2012,4,2015,4,1,0.0,1.0,0.0,1.0,0.0,1.0,0.816968,0.799343,0.479853,0.783526,

In [198]:
sub = pd.read_csv('/content/drive/MyDrive/LTFSDSFinHack3/sample_submission_ejm25Dc.csv')

In [199]:
prediction_rf.Label.value_counts()

0    11733
4      924
3      744
5      726
2      377
6      132
1      109
Name: Label, dtype: int64

In [200]:
sub['Top-up Month'] = prediction_rf.Label

In [204]:
rev_mapping_dict = {0:'No Top-up Service', 1:'12-18 Months',2:'18-24 Months', 3:'24-30 Months', 
                4:'30-36 Months',5:'36-48 Months',6:' > 48 Months'}

In [205]:
sub['Top-up Month'] = sub['Top-up Month'].replace(rev_mapping_dict) 

In [206]:
sub.to_csv('rfsmote.csv', index =False)